# Домашнее задание 5: LU- и LUP-разложение
# Якунина Е. Н., НПМбд-01-19

Импортируем необходимые библиотеки

In [1]:
import numpy as np
from scipy.linalg import lu_factor, lu_solve

Рассмотрим решение СЛАУ методом LU-разложения

Напишем функцию, которая раскладывает поданную ей квадратную матрицу на нижнетреугольную матрицу L и верхнетреугольную матрицу U

In [2]:
def LU(A):
    U = np.copy(A)
    L = np.identity(len(A))
    for i in range(len(A) - 1):
        k = U[i + 1:, i] / U[i, i]
        L[i + 1:, i] += k
        k.resize(k.size, 1)
        U[i + 1:, :] -= k * U[i]
    return L, U

Проверим функцию для квадратной матрицы 3 x 3

In [3]:
A = np.array([[2, 7, 11.], 
              [0, -2, 1.], 
              [2, 5, 3.]])

In [4]:
L, U = LU(A)
print("L = {}".format(L), "U = {}".format(U), sep = '\n')

L = [[1. 0. 0.]
 [0. 1. 0.]
 [1. 1. 1.]]
U = [[ 2.  7. 11.]
 [ 0. -2.  1.]
 [ 0.  0. -9.]]


In [5]:
A == L @ U

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

Перед написанием функции для решения СЛАУ Ax=b напишем вспомогательные функции для решения уравнений Ly=b и Ux=y

In [6]:
def Ly_b(L, b):
    n = len(L)
    y = np.zeros(n)
    y[0] = b[0] / L[0, 0]
    for i in range(1, n):
        y[i] = ( b[i] - L[i, :i] @ b[:i]) / L[i, i]
    y.resize(n, 1)
    return y

In [7]:
def Ux_y(U, y):
    n = len(U)
    x = np.zeros(n)
    x[n - 1] = y[n - 1] / U[n - 1, n - 1] 
    for i in range(n - 2, -1, -1):
        x[i] = ( y[i] - U[i, i + 1:] @ x[i + 1:]) / U[i, i]
    x.resize(n, 1)
    return x

Теперь напишем саму функцию, которая, принимая на вход матрицу и вектор-столбец, с помощью LU-разложения находит решение СЛАУ. 

In [8]:
def solver_LU(A, b):
    L, U = LU(A)
    y = Ly_b(L, b)
    x = Ux_y(U, y)
    return x

In [9]:
a = np.array([[4], 
              [10], 
              [5]])

In [10]:
solver_LU(A, a)

array([[12.25],
       [-4.5 ],
       [ 1.  ]])

Проверим, получим ли мы такой же ответ, используя функции lu_factor и lu_solve библиотеки scipy.linalg

In [11]:
lu_solve(lu_factor(A), a)

array([[12.25],
       [-4.5 ],
       [ 1.  ]])

Однако, если поменять в матрице A первую (A[0]) и вторую (A[1]) строки, то разложить такую матрицу на L и U, в которых бы находились конкретные числа, уже не получится

In [12]:
def change_str(A, i, j):
    n = np.array(A[j])
    A[j] = A[i]
    A[i] = n
    return A

In [13]:
LU(change_str(A, 0, 1))

C:\Users\katya\AppData\Local\Temp/ipykernel_9632/1737286041.py:5: RuntimeWarning: divide by zero encountered in true_divide
  k = U[i + 1:, i] / U[i, i]
C:\Users\katya\AppData\Local\Temp/ipykernel_9632/1737286041.py:8: RuntimeWarning: invalid value encountered in multiply
  U[i + 1:, :] -= k * U[i]
C:\Users\katya\AppData\Local\Temp/ipykernel_9632/1737286041.py:5: RuntimeWarning: invalid value encountered in true_divide
  k = U[i + 1:, i] / U[i, i]


(array([[ 1.,  0.,  0.],
        [inf,  1.,  0.],
        [inf, nan,  1.]]),
 array([[  0.,  -2.,   1.],
        [ nan,  inf, -inf],
        [ nan,  nan,  nan]]))

In [14]:
A

array([[ 0., -2.,  1.],
       [ 2.,  7., 11.],
       [ 2.,  5.,  3.]])

Чтобы получить матрицы L и U, исходная матрица приводится к ступенчатому виду. В LU-разложении при этом не предусматривается возможность проводить какие-либо изменения над исходной матрицей. Соответственно, если в процессе преобразования матрицы к ступенчатому виду ведущий элемент на главной диагонали матрицы оказывается равным нулю, мы не можем избежать деления на нуль. Таким образом, LU-разложение не подойдет для решения систем, в которых приведение к ступенчатому виду матрицы коэффициентов предполагает перестановку строк.

Для таких матриц подходит метод LUP-разложения

Напишем функцию, которая раскладывает поданную ей квадратную матрицу на нижнетреугольную матрицу L, верхнетреугольную матрицу U и матрицу перестановок P

In [15]:
def change_stl(A, i, j):
    A = np.transpose(A)
    A = change_str(A, i, j)
    return np.transpose(A)

In [16]:
def LUP(A):
    U = np.copy(A)
    L = np.identity(len(A))
    P = np.identity(len(A))
    for i in range(len(A) - 1):
        j = i + 1
        while U[i][i] == 0:
            change_str(U, i, j)
            change_str(P, i, j)
            change_str(L, i, j)
            change_stl(L, i, j)
            j += 1
        k = U[i + 1:, i] / U[i, i]
        L[i + 1:, i] += k
        k.resize(k.size, 1)
        U[i + 1:, :] -= k * U[i]
    return P, L, U

In [17]:
P, L, U = LUP(A)
print("P = {}".format(P), "L = {}".format(L), "U = {}".format(U), sep = '\n')

P = [[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
L = [[1. 0. 0.]
 [0. 1. 0.]
 [1. 1. 1.]]
U = [[ 2.  7. 11.]
 [ 0. -2.  1.]
 [ 0.  0. -9.]]


In [18]:
P @ A == L @ U

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

Напишем функцию, которая, принимая на вход матрицу и вектор-столбец, с помощью LUP-разложения находит решение СЛАУ.

In [19]:
def solver_LUP(A, b):
    P, L, U = LUP(A)
    b = P @ b
    y = Ly_b(L, b)
    x = Ux_y(U, y)
    return x

Поменяем соответственно строки вектор-столбца a и проверим, получим ли мы тот же ответ

In [20]:
change_str(a, 0, 1)

array([[10],
       [ 4],
       [ 5]])

In [21]:
a

array([[10],
       [ 4],
       [ 5]])

In [22]:
solver_LUP(A, a)

array([[12.25],
       [-4.5 ],
       [ 1.  ]])

Проверим, получим ли мы такой же ответ, используя функции lu_factor и lu_solve библиотеки scipy.linalg

In [23]:
lu_solve(lu_factor(A), a)

array([[12.25],
       [-4.5 ],
       [ 1.  ]])

Можно также проверить, например, матрицу

In [24]:
B = np.array([[1, 2, 3], 
              [2, 4, 5.], 
              [6, 11, -1]])

In [25]:
LU(B)

C:\Users\katya\AppData\Local\Temp/ipykernel_9632/1737286041.py:5: RuntimeWarning: divide by zero encountered in true_divide
  k = U[i + 1:, i] / U[i, i]
C:\Users\katya\AppData\Local\Temp/ipykernel_9632/1737286041.py:8: RuntimeWarning: invalid value encountered in multiply
  U[i + 1:, :] -= k * U[i]


(array([[  1.,   0.,   0.],
        [  2.,   1.,   0.],
        [  6., -inf,   1.]]),
 array([[  1.,   2.,   3.],
        [  0.,   0.,  -1.],
        [ nan,  nan, -inf]]))

При приведении матрицы B к ступенчатому виду ведущий элемент во второй строчке главной диагонали обращается в нуль, соответственно, разложение LU будет неэффективно

In [26]:
LUP(B)

(array([[1., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.]]),
 array([[1., 0., 0.],
        [6., 1., 0.],
        [2., 0., 1.]]),
 array([[  1.,   2.,   3.],
        [  0.,  -1., -19.],
        [  0.,   0.,  -1.]]))

In [27]:
P, L, U = LUP(B)
print("P = {}".format(P), "L = {}".format(L), "U = {}".format(U), sep = '\n')

P = [[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
L = [[1. 0. 0.]
 [6. 1. 0.]
 [2. 0. 1.]]
U = [[  1.   2.   3.]
 [  0.  -1. -19.]
 [  0.   0.  -1.]]


In [28]:
P @ B == L @ U

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

Уже с помощью LUP-разложения удалось представить матрицу через нижнетреугольную и верхнетреугольную

Зададим теперь также вектор-столбец и решим СЛАУ

In [29]:
b = np.array([[2], 
              [5.], 
              [-1]])

In [30]:
solver_LUP(B, b)

array([[-59.],
       [ 32.],
       [ -1.]])

Проверим, получим ли мы такой же ответ, используя функции lu_factor и lu_solve библиотеки scipy.linalg

In [31]:
lu_solve(lu_factor(B), b)

array([[-59.],
       [ 32.],
       [ -1.]])

Таким образом, можно увидеть, что метод LUP-разложения подходит для решения СЛАУ в большем количестве случаев, чем метод LU-разложения. Некоторые СЛАУ, как было показано выше, могут быть решены с помощью LU-разложения при некоторой конкретной очередности уравнений, но после перестановки строк системы матрица коэффициентов может уже не разложиться на нижне- и верхнеквадратные матрицы.